# Loading and Cleaning the data


In [2]:
import pandas as pd

edges_df = pd.read_csv(
    "data/OBS_data.txt", sep="\t", header=0, usecols=[0, 1, 2, 3, 4], parse_dates=[0]
)
edges_df["DateTime"] = pd.to_datetime(edges_df["DateTime"]).astype("datetime64[ms]")
edges_df.dropna(axis=0, inplace=True)
edges_df["Weight"] = edges_df["Category"].apply(
    lambda c: 1 if (c == "Affiliative") else (-1 if (c == "Agonistic") else 0)
)
print(edges_df.head())

              DateTime   Actor Recipient  Behavior     Category  Weight
15 2019-06-13 09:50:00  ANGELE    FELIPE  Grooming  Affiliative       1
17 2019-06-13 09:50:00  ANGELE    FELIPE  Grooming  Affiliative       1
19 2019-06-13 09:51:00  FELIPE    ANGELE   Resting  Affiliative       1
20 2019-06-13 09:51:00  FELIPE      LIPS   Resting  Affiliative       1
21 2019-06-13 09:51:00  ANGELE    FELIPE  Grooming  Affiliative       1


/var/folders/rm/nnz28pc96cv2xqzbwlwvwl2r0000gn/T/ipykernel_70748/2052837081.py:3: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  edges_df = pd.read_csv(


# Creating a graph

In [5]:
import raphtory as rp

g = rp.Graph()
g.load_edges_from_pandas(
    df=edges_df,
    src="Actor",
    dst="Recipient",
    time="DateTime",
    layer="Behavior",
    props=["Weight"],
)
print(g)


Graph(number_of_edges=290, number_of_vertices=22, number_of_temporal_edges=3196, earliest_time="1560419400000", latest_time="1562756700000")


# Basic Metrics


In [6]:
print("Stats on the graph structure:")

number_of_vertices = g.count_vertices()
number_of_edges = g.count_edges()
total_interactions = g.count_temporal_edges()
unique_layers = g.unique_layers

print("Number of vertices (Baboons):", number_of_vertices)
print("Number of unique edges (src,dst,layer):", number_of_edges)
print("Total interactions (edge updates):", total_interactions)
print("Unique layers:", unique_layers, "\n")


print("Stats on the graphs time range:")

earliest_datetime = g.earliest_date_time
latest_datetime = g.latest_date_time
earliest_epoch = g.earliest_time
latest_epoch = g.latest_time

print("Earliest datetime:", earliest_datetime)
print("Latest datetime:", latest_datetime)
print("Earliest time (Unix Epoch):", earliest_epoch)
print("Latest time (Unix Epoch):", latest_epoch)

Stats on the graph structure:
Number of vertices (Baboons): 22
Number of unique edges (src,dst,layer): 290
Total interactions (edge updates): 3196
Unique layers: ['_default', 'Grooming', 'Resting', 'Presenting', 'Playing with', 'Grunting-Lipsmacking', 'Supplanting', 'Threatening', 'Submission', 'Touching', 'Avoiding', 'Attacking', 'Carrying', 'Embracing', 'Mounting', 'Copulating', 'Chasing'] 

Stats on the graphs time range:
Earliest datetime: 2019-06-13 09:50:00
Latest datetime: 2019-07-10 11:05:00
Earliest time (Unix Epoch): 1560419400000
Latest time (Unix Epoch): 1562756700000


# Update history

In the code below we create a vertex object for the monkey Felipe and see when their updates occurred. 


In [7]:
v = g.vertex("FELIPE")
print(
    f"{v.name}'s first interaction was at {v.earliest_date_time} and their last interaction was at {v.latest_date_time}\n"
)
print(f"{v.name} had interactions at the following times: {v.history()}\n")

FELIPE's first interaction was at 2019-06-13 09:50:00 and their last interaction was at 2019-07-10 11:05:00

FELIPE had interactions at the following times: [1560419400000, 1560419460000, 1560419520000, 1560419580000, 1560419640000, 1560420720000, 1560421260000, 1560422580000, 1560423360000, 1560423420000, 1560423600000, 1560437400000, 1560437460000, 1560437640000, 1560439920000, 1560439980000, 1560440160000, 1560440640000, 1560441780000, 1560441840000, 1560441960000, 1560443040000, 1560504000000, 1560504540000, 1560506760000, 1560507540000, 1560508320000, 1560508380000, 1560508500000, 1560523260000, 1560523620000, 1560523680000, 1560523740000, 1560523860000, 1560524880000, 1560525600000, 1560525720000, 1560526140000, 1560526260000, 1560526320000, 1560769320000, 1560769440000, 1560775740000, 1560780060000, 1560780120000, 1560853500000, 1560854160000, 1560854400000, 1560855660000, 1560865320000, 1560865440000, 1560865500000, 1560934560000, 1560934680000, 1560934740000, 1560935460000, 15

# Neighbours, edges and paths


In [8]:
v = g.vertex("FELIPE")
v_name = v.name
in_degree = v.in_degree()
out_degree = v.out_degree()
in_edges = v.in_edges
neighbours = v.neighbours
neighbour_names = v.neighbours.name.collect()

print(
    f"{v_name} has {in_degree} incoming interactions and {out_degree} outgoing interactions.\n"
)
print(in_edges)
print(neighbours, "\n")
print(f"{v_name} interacted with the following baboons {neighbour_names}")


FELIPE has 17 incoming interactions and 18 outgoing interactions.

Edges(Edge(source=ANGELE, target=FELIPE, earliest_time=1560419400000, latest_time=1562753640000, properties={Weight: 1}), Edge(source=LIPS, target=FELIPE, earliest_time=1560423600000, latest_time=1562756700000, properties={Weight: 1}), Edge(source=NEKKE, target=FELIPE, earliest_time=1560443040000, latest_time=1562596380000, properties={Weight: 1}), Edge(source=LOME, target=FELIPE, earliest_time=1560421260000, latest_time=1562149080000, properties={Weight: 1}), Edge(source=BOBO, target=FELIPE, earliest_time=1560423360000, latest_time=1561543080000, properties={Weight: -1}), Edge(source=ATMOSPHERE, target=FELIPE, earliest_time=1560524880000, latest_time=1561638540000, properties={Weight: 1}), Edge(source=FEYA, target=FELIPE, earliest_time=1560853500000, latest_time=1562586000000, properties={Weight: 1}), Edge(source=FANA, target=FELIPE, earliest_time=1560526140000, latest_time=1562752800000, properties={Weight: 1}), Edge(

# Chaining Functions 

We can see a basic example of chaining function chains to get the names of all the monkeys and the names of their two-hop neighbours.

In [9]:
vertex_names = g.vertices.name
two_hop_neighbours = g.vertices.neighbours.neighbours.name.collect()
combined = zip(vertex_names, two_hop_neighbours)
for name, two_hop_neighbour in combined:
    print(f"{name} has the following two hop neighbours {two_hop_neighbour}") 

ANGELE has the following two hop neighbours ['ANGELE', 'LIPS', 'NEKKE', 'LOME', 'BOBO', 'ATMOSPHERE', 'FEYA', 'FANA', 'PIPO', 'MUSE', 'MAKO', 'MALI', 'PETOULETTE', 'ARIELLE', 'HARLEM', 'VIOLETTE', 'EWINE', 'SELF', 'ANGELE', 'FELIPE', 'NEKKE', 'LOME', 'BOBO', 'ATMOSPHERE', 'FEYA', 'FANA', 'PIPO', 'KALI', 'MUSE', 'MAKO', 'MALI', 'PETOULETTE', 'ARIELLE', 'HARLEM', 'VIOLETTE', 'EWINE', 'ANGELE', 'FELIPE', 'LIPS', 'LOME', 'BOBO', 'ATMOSPHERE', 'FEYA', 'FANA', 'PIPO', 'KALI', 'MUSE', 'MAKO', 'MALI', 'PETOULETTE', 'ARIELLE', 'HARLEM', 'VIOLETTE', 'EWINE', 'ANGELE', 'FELIPE', 'LIPS', 'NEKKE', 'BOBO', 'ATMOSPHERE', 'FEYA', 'FANA', 'PIPO', 'KALI', 'MUSE', 'MAKO', 'MALI', 'PETOULETTE', 'ARIELLE', 'HARLEM', 'VIOLETTE', 'EWINE', 'ANGELE', 'FELIPE', 'LIPS', 'NEKKE', 'LOME', 'ATMOSPHERE', 'FEYA', 'FANA', 'PIPO', 'KALI', 'MUSE', 'MAKO', 'MALI', 'PETOULETTE', 'ARIELLE', 'HARLEM', 'VIOLETTE', 'EWINE', 'ANGELE', 'FELIPE', 'LIPS', 'NEKKE', 'LOME', 'BOBO', 'FEYA', 'FANA', 'PIPO', 'KALI', 'MUSE', 'MAKO', 'M

# Chains with properties 

Finding the most annoying monkey by ranking globally who on average has had the most negative interactions initiated against them.


In [11]:
v = g.vertex("FELIPE")
neighbours_weighted = list(
    zip(
        v.out_edges.dst.name,
        v.out_edges.properties.temporal.get("Weight").values().sum(),
    )
)
sorted_weights = sorted(neighbours_weighted, key=lambda v: v[1], reverse=True)
print(f"Felipe's favourite baboons in descending order are {sorted_weights}")

annoying_monkeys = list(
    zip(
        g.vertices.name,
        g.vertices.in_edges.properties.temporal.get("Weight")
        .values()
        .sum()  # sum the weights within each edge
        .mean()  # average the summed weights for each monkey
        .collect(),
    )
)
most_annoying = sorted(annoying_monkeys, key=lambda v: v[1])[0]
print(
    f"{most_annoying[0]} is the most annoying monkey with an average score of {most_annoying[1]}"
)


Felipe's favourite baboons in descending order are [('NEKKE', 41), ('ANGELE', 31), ('MAKO', 26), ('LOME', 23), ('LIPS', 11), ('HARLEM', 10), ('FANA', 8), ('MALI', 6), ('FEYA', 5), ('ARIELLE', 5), ('EWINE', 5), ('PIPO', 3), ('SELF', 2), ('BOBO', 1), ('ATMOSPHERE', 1), ('PETOULETTE', 1), ('VIOLETTE', 1), ('MUSE', -1)]
EXTERNE is the most annoying monkey with an average score of -2.0
